In [ ]:
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib
matplotlib.rcParams['animation.embed_limit']=200
import glob

sys.path.append("../../../../src")
from visualize import vals_anime

In [ ]:
def sort_legend(ax):
    handles, labels = ax.get_legend_handles_labels()
    handles = [handles[i] for i in np.argsort(labels)]
    labels = np.sort(labels)
    ax.legend(handles, labels)

def vis_LRlim(data_dirs, nframes=2000, title=None):
    
    fig, ax = plt.subplots(2, len(data_dirs), figsize=(16,8))
        
    animes = []
    
    for i, data_dir in enumerate(sorted(data_dirs)):        
        for j, fname in enumerate(sorted(glob.iglob(f"{data_dir}/*"))):
            f = open(fname,'rb')
            stat = pickle.load(f)
            f.close()
            bn = fname.split("_")[-1].split(".")[0]
            lr = stat["lr"]
            loss = stat["loss"]
            acc = stat["acc"]
            animes.append(vals_anime(loss[:nframes], ax[0,i], f"loss_{bn}"))
            animes.append(vals_anime(acc[:nframes], ax[1,i], f"acc_{bn}"))
            animes[-2].ln.set_label(lr)
            animes[-1].ln.set_label(lr)
            
    for i in range(len(data_dirs)):
        sort_legend(ax[0,i])
        sort_legend(ax[1,i])
            
    def init():
        for anime in animes:
            anime.init()
        fig.suptitle(f"[{title}] i=0")
        return

    def update(i):
        for anime in animes:
            anime.update(i)        
        fig.suptitle(f"[{title}] i={i}")
        return

    ani = FuncAnimation(fig, update, init_func=init, frames=nframes,
                        interval=50)
    ani = ani.to_jshtml()
    #ani = ani.to_html5_video()
    plt.close()
    return ani#HTML(ani)


In [ ]:
data_dirs = glob.glob("loss_acc_lr/*")

ani_LRlim = vis_LRlim(data_dirs, nframes=20, title="LR limitation")

In [ ]:
HTML(ani_LRlim)